In [1]:
import glob
import skimage
import skimage.io as sio
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import skimage.feature as feature
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny, local_binary_pattern
import os
from skimage.filters import gabor_kernel
import time
from multiprocessing import Pool

# This is where there will be a simple example to load and run the final classifier

# This is where there will be an example of loading features and training the classifier, and some quanitification and visualization of error

# This is where there will be detailed explanation of features and code to extract all of them.

In [2]:
#get the names of folders and alphabetize so that it's universal across machines
folds = glob.glob('50_categories/*')
folds.sort()
#the order images of within folders doesn't matter
#get all the individual file names
ims_by_category = [glob.glob(f + '/*') for f in folds]
#unravel the file names into a single vector
paths = [path for item in ims_by_category[0:50] for path in item ]

#create a vector with a numerical label for every image 
folds_labels = []
for i in range(0,50):
    folds_labels.append(np.repeat([i], len(ims_by_category[i])))
folds_labels = np.concatenate(folds_labels[0:50], axis =0 )

#grab the names of each category for visualization
categories_names = [os.path.split(s)[1] for s in folds]

In [3]:
def center_crop(image, new_width, new_height):
    """crop the image to size new_width, new_height, taking the center of the image"""
    height = np.shape(image)[0]
    width = np.shape(image)[1]
    left = width/2 - new_width/2
    top = height/2 - new_height/2
    right = width/2 + new_width/2
    bottom = height/2 + new_height/2
    if len(image.shape) > 2:
        return image[int(top):int(bottom), int(left):int(right),:]
    return image[int(top):int(bottom), int(left):int(right)]

def set_center_zero(image, new_width, new_height):
    """set the center of the image, as defined by new_width, new_height, to 0. Used for color features"""
    image = image.copy()
    height = np.shape(image)[0]
    width = np.shape(image)[1]
    left = width/2 - new_width/2
    top = height/2 - new_height/2

    right = width/2 + new_width/2
    bottom = height/2 + new_height/2
    image[int(top):int(bottom), int(left):int(right),:] = 0
    return image
   
def get_sized_image(im, final_size, preserve_aspect, do_downscale):
    im = skimage.color.rgb2grey(im)
    
    if do_downscale:
        if preserve_aspect:
            transform=int(min(np.shape(im)[0], np.shape(im)[1])/final_size)
            transform = (transform, transform)
        if not preserve_aspect:
            transform = (int(np.shape(im)[0]/final_size), int(np.shape(im)[1]/final_size))
        im2 = skimage.transform.downscale_local_mean(im, transform)

        #this won't be exact, so crop to ensure
        im2 = center_crop(im2, final_size, final_size)
            
    else:
        if preserve_aspect:
            smallest_dim = min(np.shape(im)[0], np.shape(im)[1])
            scale_fact = final_size/smallest_dim
            new_dim = (int(im.shape[0]*scale_fact), int(im.shape[1]*scale_fact))
            im2 = skimage.transform.resize(im, (new_dim))
            im2 = center_crop(im2, final_size,final_size)
        else:
            im2 = skimage.transform.resize(im, (final_size, final_size))
            
    return im2

In [4]:
def get_n_circ_normed(image):
    """get the number of hough circles detected in the image, divided by the total size of the image"""
    if len(np.shape(image)) > 2:
        image = skimage.color.rgb2grey(image)
    edges = canny(image, sigma=3)
    # detect series of radii based on size of image
    min_rad = int(min(image.shape)/10)
    max_rad = int(min(image.shape)/1.5)
    hough_radii = np.arange(min_rad, max_rad, 6)
    hough_res = hough_circle(edges, hough_radii)
    return len(np.where(np.ravel(hough_res) > .3)[0])/len(np.ravel(image))

def get_n_pix_per_feat(im):
    """Using the shape index, detect the number of pixels belonging to each features.
    Return the percent of pixels belonging to each of those shape features."""
    s=feature.shape_index(im)
    s=np.ravel(s)
    #shape types
    feats=[]
    #basically this gets the number of pixels in each feature, I'm not sure why the numbers are what they are
    for i in range(0, 9):
        feats.append(len([k for k in s if k >= -1+2/8*i and k< -1+2/8*(i+1)])/len(s))
    return(feats)

def get_lbp_percents(im):
    """Using the local binary pattern algorithm, determine the percent of pixels in an image
    that are either edges or corners"""
    radius = 3
    n_points = radius*8
    w = width = radius - 1
    #based on skimage tutorials, find what numbers will correspond to each category of points
    edge_labels = range(n_points // 2 - w, n_points // 2 + w + 1)
    flat_labels = list(range(0, w + 1)) + list(range(n_points - w, n_points + 2))
    i_14 = n_points // 4            # 1/4th of the histogram
    i_34 = 3 * (n_points // 4)      # 3/4th of the histogram
    corner_labels = (list(range(i_14 - w, i_14 + w + 1)) +
                     list(range(i_34 - w, i_34 + w + 1)))

    lbp=local_binary_pattern(skimage.color.rgb2grey(im), n_points, radius, 'uniform')
    n_edge = len([x for item in lbp for x in item if x in edge_labels])
    n_flat = len([x for item in lbp for x in item if x in flat_labels])
    n_corner = len([x for item in lbp for x in item if x in corner_labels])
    per_edge = n_edge /(len(np.ravel(lbp)))
    per_corner = n_corner /len(np.ravel(lbp))
    return(per_edge, per_corner)

def get_rel_n_keypoints_and_avg_scale(im):
    """Using the CENSURE feature detector, find the # of keypoints in an image, and their means scale
    Divide by total image size to provide a relative measure."""
    c=feature.CENSURE()
    c.detect(im)
    imsz = len(np.ravel(im))
    return(np.shape(c.keypoints)[0]/imsz, np.mean(c.scales)/imsz)


def get_glcm_properties(image):
    """get the gray level co-occurence matrix properties of dissimilarity, correlation, and energy
    for an image. Compute for distance determined by the size of the image and angle 0.
    
    Input: a greyscale image
    Returns: a list of length 3"""
    #the image must be converted to integers from floats for the glcm function
    dist = int(min(image.shape)/6)
    glcm = feature.greycomatrix((image*255).astype('uint32'), [dist], [0], 256, symmetric=True, normed=True)
    return [feature.greycoprops(glcm, 'dissimilarity')[0, 0], feature.greycoprops(glcm, 'correlation')[0, 0],
            feature.greycoprops(glcm, 'energy')[0, 0]]


def get_color_feats(im, do_hsv, do_cent_surround=False, cent_scale = None, ratio=False):
    """gets the mean of each color for whole image, or for 'center' and 'surround', based on rgb or hsv colors"""
    if len(im.shape) < 3:
        if ratio or not do_cent_surround:
            return np.zeros((1,3))
        else:
            return np.zeros((1,6))
    if do_hsv:
        im = skimage.color.rgb2hsv(im)
    if do_cent_surround:
        feats = np.empty((1,6))
        cent_size = max(im.shape[0], im.shape[1])*cent_scale
        #get rgb values of background, the center is set to zero, take the sum along the different axes
        #and then divide by total n pixels in background to get mean
        bgim = set_center_zero(im, cent_size, cent_size)
        n_bg_pix = len(np.ravel(im)) - cent_size*cent_size
        feats[0,0:3] = np.sum(np.sum(bgim, axis = 1), axis=0)/n_bg_pix
        #and the overall 
        cim = center_crop(im,cent_size, cent_size);
        feats[0,3:6] = np.mean(np.mean(cim,axis = 1), axis=0)
        if ratio:
            return feats[0,0:3]/feats[0,3:6]
        return feats
    else:
        return np.mean(np.mean(im, axis = 1), axis=0)
    
def get_advanced_hog_feats(fd, n_orientations):
    """given the output of a call to histogram of oriented gradients (hog), generate some summary features
    that describe the nature of orientations in the image.
    Finds the dominant orientation at each location by finding which index in the fourth dimension, which
    represents strength at different orientations, is the largest. Then it finds the overall dominant orientation
    in the image based which occurs most. Use this in calculating several 
    
    Input: fd the non-flattened output of the hog function, and n_orientations, the number of orientations
    used to calculate fd
    
    Output: a 0x6 numpy array consisting of: the most common dominant orientation (mco),
    the percent of locations where mco is the dominant orientation, var in dominant orientation, 
    total strength at the dominant orientation at each location, the strength of the orientation at locations
    where the dominant orientation is the mco divided by the total strength, and the variance in strength
    of orientedness.
    """
    X=np.empty((1,6))
    #the dominant orientation at each location
    dom_at_each_loc = np.ravel(np.argmax(fd, axis = 4))
    #count the number of times each orientation is the dominant orientation
    counts = np.bincount(dom_at_each_loc, minlength=n_orientations)
    #get the most common dominant orientation of the image
    X[0,0] = np.argmax(counts)
    #the percent of the image where that orientation is dominant
    X[0,1] = counts[np.argmax(counts)]/np.sum(counts)
    #the variance in the dominant orien
    X[0,2] = np.var(dom_at_each_loc)
    #the strength of the orientedness at the dominant orientation at each point
    strength_at_dom_or = np.ravel(np.max(fd, axis = 4))
    #the overall strength of the orientedness at dominant orientations
    X[0,3] = np.sum(strength_at_dom_or)
    #the ratio of strength at the overall dominant orient to the dominant orientations as a whole
    X[0, 4] = np.sum(strength_at_dom_or[np.where(dom_at_each_loc==np.argmax(counts))])/X[0,3]
    #the variance in strength of orientedness across the image
    X[0, 5] = np.var(strength_at_dom_or)
    return X


In [23]:
def get_shape_feats(im):
    """generates this weird set of features i've picked out. Subtracts mean of image. """
    feats = np.empty((1,14))
    #because lbp and the shape are especially slow, downsize images for them
    if np.max(im.shape) > 250:
        while np.max(im.shape) > 250:
            im = skimage.transform.rescale(im, .5)
    subim = im - np.mean(np.ravel(im))
    feats[0,0:2] = get_lbp_percents(subim) #the percent of pixels that are edges, and the percent that are corners
    feats[0,2:4] = get_rel_n_keypoints_and_avg_scale(subim) 

    feats[0,4] = get_n_circ_normed(subim) #number of circles

    feats[0, 5:14] = get_n_pix_per_feat(subim) #number of pixels falling into a bunch of wild different features
    return feats

def get_texture_feats(im):
    """don't subtract because these have their own normalization methods"""
    feats = np.empty((1,6+3))
    px_per_cell = (int(im.shape[0]/10), int(im.shape[1]/10))

    fd = feature.hog(im, orientations=8,
                                        pixels_per_cell=px_per_cell,
                     cells_per_block=(1, 1),
                                        visualise = False, feature_vector=False,
                    block_norm = 'L2-Hys')
    feats[0, 0:6] = get_advanced_hog_feats(fd, 8)
    
    feats[0,6:9] = get_glcm_properties(im)
    return feats

def get_all_color_feats(im):
    feats = np.empty((1,3+3+3))
    feats[0,0:3] = get_color_feats(im, do_hsv=False)
    feats[0,3:6] = get_color_feats(im, do_hsv=False, do_cent_surround=True, cent_scale=.5, ratio=True)
    feats[0,6:9] = get_color_feats(im, do_hsv=True)
    return feats
    
def get_one_row_feats(path):
    feats = np.empty((1, 33))
    image = sio.imread(path)
    #if the image is too large, resize it to save time
    #if np.max(image.shape) > 400:
    #    while np.max(image.shape) > 400:
    #        image = skimage.transform.rescale(image, .5)
    feats[0,0:9] = get_all_color_feats(image)
    grey = skimage.color.rgb2grey(image)
    feats[0,9:23] = get_shape_feats(grey)
    feats[0,23:32] = get_texture_feats(grey)
    return feats


# Extract and then save features

In [81]:
groups_of_paths = []
for i in range(0, int(len(paths)/250)+1):
    #if the index goes out of range it only adds the existing paths
    groups_of_paths.append(paths[i*250:(i+1)*250])
    
pool = Pool(2)
features = pool.map(gen_several_rows, groups_of_paths)

/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/feature/corner.py:373: RuntimeWarning: divide by zero encountered in true_divide
  return (2.0 / np.pi) * np.arctan((l2 + l1) / (l2 - l1))
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/feature/corner.py:373: RuntimeWarning: invalid value encountered in true_divide
  return (2.0 / np.pi) * np.arctan((l2 + l1) / (l2 - l1))
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/feature/

KeyboardInterrupt: 

/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/feature/corner.py:373: RuntimeWarning: divide by zero encountered in true_divide
  return (2.0 / np.pi) * np.arctan((l2 + l1) / (l2 - l1))
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/feature/corner.py:373: RuntimeWarning: invalid value encountered in true_divide
  return (2.0 / np.pi) * np.arctan((l2 + l1) / (l2 - l1))
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/feature/

In [24]:
start = time.time()
feats = np.empty((len(paths),15+9+9))
for i, path in enumerate(paths):
    if i%50==0:
        print(i)
        print(time.time()-start)
    feats[i,:] = get_one_row_feats(path)

0
0.010197639465332031


/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/feature/corner.py:373: RuntimeWarning: divide by zero encountered in true_divide
  return (2.0 / np.pi) * np.arctan((l2 + l1) / (l2 - l1))
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/skimage/feature/corner.py:373: RuntimeWarning: invalid value encountered in true_divide
  return (2.0 / np.pi) * np.arctan((l2 + l1) / (l2 - l1))


50
8.330198764801025
100
21.58578372001648
150
31.391534566879272
200
40.152687072753906
250
49.18550944328308
300
58.38654923439026
350
67.13288807868958
400
76.01668429374695


/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


450
84.88701677322388
500
93.63058567047119
550
103.9878921508789
600
119.61756682395935
650
144.16669034957886
700
161.32083249092102
750
179.50718212127686
800
199.56450510025024
850
214.93493604660034
900
229.4680073261261
950
244.47488164901733
1000
264.8129801750183
1050
281.14953780174255
1100
294.8726952075958
1150
310.31071949005127
1200
323.21762442588806
1250
346.3688313961029
1300
365.2730944156647
1350
381.90933775901794
1400
399.4136152267456
1450
419.29518008232117
1500
443.889714717865
1550
468.51666164398193
1600
485.3014123439789
1650
502.4419116973877
1700
521.1120388507843
1750
541.7718617916107
1800
563.6794879436493
1850
580.3476150035858
1900
595.4143559932709
1950
611.6585104465485
2000
623.4983816146851
2050
635.573296546936
2100
650.1364402770996
2150
665.7307486534119
2200
681.2772564888
2250
698.5296347141266
2300
720.4767966270447
2350
738.7786462306976
2400
754.5950238704681
2450
766.6594247817993
2500
783.4675796031952
2550
798.6464593410492
2600
811.05661

In [25]:
np.savez('finalish_features.npz', feats)

## If I end up using BOW, maybe a subsection on it alone